In [1]:
#Financial Model Project 
#Author: Tom Pelletier 

import requests as r
import pandas as pd 
import numpy as np 
import json
import time

In [2]:
#Data from API https://financialmodelingprep.com/developer/docs/

In [4]:
#Get Jsons of Company Financials
company_code = 'AAPL'

get_profile = r.get('https://financialmodelingprep.com/api/v3/company/profile/%s' % (company_code)).json()
get_Qincome = r.get('https://financialmodelingprep.com/api/v3/financials/income-statement/%s?period=quarter' % (company_code)).json()
get_Qbalance = r.get('https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/%s?period=quarter' % (company_code)).json()
get_Qcashflow = r.get('https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/%s?period=quarter' % (company_code)).json()
get_Qgrowth = r.get('https://financialmodelingprep.com/api/v3/financial-statement-growth/%s?period=quarter' % (company_code)).json()

#all company symbols on NASDAQ
get_symbols = r.get('https://financialmodelingprep.com/api/v3/company/stock/list').json()
symbol_lst = [get_symbols['symbolsList'][i]['symbol'] for i in range(len(get_symbols['symbolsList']))]

all_jsons = [get_Qincome,get_Qbalance,get_Qcashflow,get_Qgrowth]

In [5]:

#Convert Json features and label to Pandas DataFrames WRANGLING

def json_df():
    df_lst = []

    #make individual DFs and store them in list
    for i in all_jsons: 
        key = list(i.keys())[1]
        df = pd.DataFrame.from_dict(i[key])
        df_lst.append(df)
    df = pd.DataFrame.from_dict(get_profile)
    
    #all we want from the profile json is the sector of the company
    sector = pd.DataFrame.from_dict(get_profile).T['sector'][1]
    
    #merge DataFrames into one
    df = df_lst[0].merge(df_lst[1])
    df = df.merge(df_lst[2])
    df = df.merge(df_lst[3])
    
    #add the sector column
    df['Sector'] = sector
    
    #save and drop date column
    quarters = list(df['date'])
    df.drop('date',inplace=True,axis=1)
    
    #convert dtypes to numeric
    cols = df.columns
    for i in cols: 
        if i != 'Sector':
            df[i] = pd.to_numeric(df[i])
            
    #deal with the label(price) column:
    Mean_Q_P = []
    Mean_Q_V = []
    
    for i in range(len(quarters)): 
        if i <39:
            close_lst = []
            vol_lst = []
            start = quarters[i] 
            end = quarters[i+1]
            get_price = r.get('https://financialmodelingprep.com/api/v3/historical-price-full/%s?from=%s&to=%s' % (company_code,end,start)).json()

            for i in get_price['historical']:
                close = i['close']
                volume = i['volume']
                close_lst.append(close)
                vol_lst.append(volume)

            Mean_Q_P.append(np.mean(close_lst))
            Mean_Q_V.append(np.mean(vol_lst))
    #remove nan
    filtered_P = [i for i in Mean_Q_P if str(i)!='nan']
    filtered_V = [i for i in Mean_Q_V if str(i)!='nan']
    df_y = pd.DataFrame([filtered_P,filtered_V]).T
    df_y.columns = ['Price','Volume']
    #match the shape of both DFs
    df = df[df.index <len(df_y)]
    df['Volume'] = df_y['Volume']
    df_y.drop('Volume',inplace=True,axis=1)
    
    return df, df_y

#get all financials froma all publicly listed companies
def Total_Nasdaq_Financials(lst=symbol_lst):
    counter = 0
    for i in lst:
        counter+=1
        company_code = i
        
        if counter == 1:
            df, df_y = json_df()
        
        df_, df_y_ = json_df()
        df = pd.concat([df, df_])
        df_y = pd.concat([df_y, df_y_])
        
    return df, df_y
        
        
        
        
        

In [6]:
five = symbol_lst[:10]

In [7]:
t0 = time.time()

df, df_y = Total_Nasdaq_Financials(five)

t1 = time.time()
T = (t1-t0)/10
print(T)

/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwa

22.166338396072387


In [8]:
(T*8000)/60

2955.511786142985

In [9]:
2955.746/60

49.262433333333334